In [1]:
import os
import pandas as pd 
import numpy as np
from gensim.models import word2vec
from gensim.models import Doc2Vec, doc2vec
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from gensim.similarities.index import AnnoyIndexer
import utils
import time 
from itertools import chain
from collections import Counter
from sklearn.multiclass import OneVsRestClassifier
import multiprocessing
from sklearn.cross_validation import train_test_split
import operator
import pickle
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import random
from sklearn.ensemble import ExtraTreesClassifier
%matplotlib inline

In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
utils.data_directory()

On AWS instance
/home/ec2-user/data


In [4]:
aisles = pd.read_csv("aisles.csv")
departements = pd.read_csv("departments.csv")
order_products__prior = pd.read_csv("order_products__prior.csv")
order_products__train = pd.read_csv("order_products__train.csv")
order = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [5]:
order.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [6]:
liste_user = np.unique(order.ix[order["eval_set"]=="train", "user_id"])

In [17]:
merge_order_product = pd.merge(order_products__prior[["order_id", "product_id", "reordered"]], 
                               order[["order_id", "user_id"]], on="order_id")
merge_order_product = merge_order_product[merge_order_product["reordered"]==1]

In [22]:
order_products__train = pd.merge(order_products__train, order[["order_id", "user_id"]],how="left", on="order_id")
order_products__train["product_id"] = order_products__train["product_id"].astype("unicode")
order_products__train = order_products__train[order_products__train["reordered"]==1]

In [18]:
merge_order_product.shape

(19126536, 4)

In [19]:
merge_order_product.groupby(["user_id","order_id"])["product_id"].apply(list)

user_id  order_id
1        431534                      [196, 12427, 10258, 25133, 13176]
         473747                                    [196, 12427, 10258]
         550135                      [196, 10258, 12427, 25133, 13032]
         2254736                     [196, 12427, 10258, 25133, 26405]
         2295261              [49235, 46149, 25133, 196, 10258, 12427]
         2398795                                   [196, 12427, 26088]
         2550362              [196, 46149, 25133, 10258, 13032, 12427]
         3108588                            [12427, 196, 10258, 25133]
         3367565                            [196, 12427, 10258, 25133]
2        738281                                                [32792]
         788338      [32792, 24852, 34688, 48110, 47209, 32052, 177...
         839880      [24852, 16589, 1559, 19156, 18523, 33754, 2170...
         1199898     [32792, 12000, 9681, 32139, 27344, 24852, 4506...
         1402090     [32792, 19051, 19156, 18523, 1559, 221

In [24]:
order_products__train[order_products__train["user_id"]==1]

,order_id,product_id,add_to_cart_order,reordered,user_id
484420,1187899,196,1,1,1
484421,1187899,25133,2,1,1
484422,1187899,38928,3,1,1
484423,1187899,26405,4,1,1
484424,1187899,39657,5,1,1
484425,1187899,10258,6,1,1
484426,1187899,13032,7,1,1
484427,1187899,26088,8,1,1
484429,1187899,49235,10,1,1
484430,1187899,46149,11,1,1
